# Reinforcement Learning in Finite MDPs

In [0]:
!git clone https://github.com/rlgammazero/mvarl_hands_on.git > /dev/null 2>&1

## MDPs

In [0]:
import sys
sys.path.insert(0, './mvarl_hands_on/utils')
import numpy as np
from scipy.special import softmax # for SARSA
import matplotlib.pyplot as plt
import json
import math
from cliffwalk import CliffWalk
from test_env import ToyEnv1

Setting up the environment

In [0]:
env = CliffWalk(proba_succ=0.98)

####################################################################################
# You probably want to test smaller enviroments before
#env = ToyEnv1(gamma=0.95)
####################################################################################

# Useful attributes
print("Set of states:", env.states)
print("Set of actions:", env.actions)
print("Number of states: ", env.Ns)
print("Number of actions: ", env.Na)
print("P has shape: ", env.P.shape)  # P[s, a, s'] = env.P[s, a, s']
print("discount factor: ", env.gamma)
print("")

# Usefult methods
state = env.reset() # get initial state
print("initial state: ", state)
print("reward at (s=1, a=3,s'=2): ", env.reward_func(1,3,2))
print("")

# A random policy
policy = np.random.randint(env.Na, size = (env.Ns,))
print("random policy = ", policy)

# Interacting with the environment
print("(s, a, s', r):")
for time in range(4):
    action = policy[state]
    next_state, reward, done, info = env.step(action)
    print(state, action, next_state, reward)
    if done:
        break
    state = next_state
print("")
print(env.R.shape)

## Question 1: Value iteration
1. Write a function applying the optimal Bellman operator on a provided Q function: $Q_1 = LQ_0, \; Q_0\in \mathbb{R}^{S\times A}$
2. Write a function implementing Value Iteration (VI) with $\infty$-norm stopping condition (reuse function implemented in 1)
3. Evaluate the convergence of your estimate, i.e., plot the value $\|Q_n - Q^\star\|_{\infty} = \max_{s,a} |Q_n(s,a) - Q^\star(s,a)|$

In [0]:
# --------------
# Point 1
# --------------
def bellman_operator(Q0, Ns, Na, R, P, gamma):
    "Ns: Number of states: "
    "Na: Number of actions: "
    "P is P[s,a,s'] "
    "R: reward R[s,a,s']"
    greedy_policy = np.argmax(Q0 , axis=1)
    Q1 = np.sum(R*P,axis = 2) + gamma* np.sum(P * Q0[np.arange(len(Q0)),greedy_policy] , axis = 2)
    greedy_policy = np.argmax(Q1 , axis=1)
    return Q1, greedy_policy

#    r = np.zeros(shape = (Ns,Na))
#    greedy_policy = []
#    g = 0
#    for s in range(Ns):
#      for a in range(Na):
#        for sp in range(Ns):
#          r[s,a] += R[s,a,sp]*P[s,a,sp] 
#    Q1 = np.zeros(shape = (Ns,Na))
#    for s in range(Ns):      
#      for a in range(Na):
#        val=0
#        for sp in range(Ns):
#          val += gamma*P[s,a,sp]*Q0[s,a]
#        val += r[s,a]
#        if val > Q1[s,a]:
#          Q1[s,a] = val
#          greedy= a
#        greedy_policy.append(greedy)
#      g += 1
#    return Q1, greedy_policy
    

In [0]:
# --------------
# Point 2
# --------------

def value_iteration(Q0, env, epsilon=1e-5):
    
    gamma = env.gamma
    k = 1
    Q_history = [Q0]
    Q_history.append( bellman_operator(Q0, env.Ns, env.Na, env.R, env.P, gamma)[0])
    while np.max(np.linalg.norm(Q_history[k]-Q_history[k-1], ord = np.inf)) > epsilon:    
      Q, greedy_policy = bellman_operator(Q_history[-1], env.Ns, env.Na, env.R, env.P, gamma)
      Q_history.append(Q)
      k+=1  
    return Q, greedy_policy, Q_history



In [0]:
# --------------
# Point 3
# --------------
with open("./mvarl_hands_on/data/Q_opts.json", "r") as fp:
    Qopts = json.load(fp)
Qstar = Qopts["{}_{}".format(type(env).__name__,env.gamma)]


Q0 = np.zeros((env.Ns,env.Na))
Q, greedy_policy, Q_history = value_iteration(Q0, env, epsilon=1e-5)
iteration = np.arange(len(Q_history)-1)
norm_values = []
for k in range(1, len(Q_history)):
  norm_values.append(np.linalg.norm(Q_history[k]-Qstar, ord = np.inf))

plt.plot(norm_values)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Q-learning: Convergence of Q")

In [0]:
state = env.reset()
env.render()
for i in range(50):
    action = greedy_policy[state]
    state, reward, done, _ = env.step(action)
    env.render()

## Question 2: Q learning
Q learning is a model-free algorithm for estimating the optimal Q-function online.
It is an off-policy algorithm since the samples are collected with a policy that is (potentially) not the one associated to the estimated Q-function.

1. Implement Q learning with $\epsilon$-greedy exploration.
  - Plot the error in Q-functions over iterations
  - Plot the sum of rewards as a function of iteration


$\epsilon$-greedy policy:
$$
\pi(s) = \begin{cases}
\max_a Q(s,a) & \text{w.p.} \epsilon\\
\text{random action} & \text{w.p.} 1- \epsilon
\end{cases}
$$

In [0]:
# ---------------------------
# Q-Learning
# ---------------------------

class QLearning:
    """
    Q learning with epsilon-greedy exploration
    """
    def __init__(self,env, epsilon):
        self.Ns, self.Na = env.Ns, env.Na
        self.gamma = env.gamma
        self.epsilon = epsilon
        self.Q = np.zeros((self.Ns, self.Na))
        self.N = np.zeros((self.Ns, self.Na))
    
    def sample_action(self, state, greedy):
        p = np.random.rand(1)
        if p > self.epsilon:
          return greedy[state]
        else:
          return np.random.randint(self.Na)
    
    def update(self, state, action, next_state, reward):
        self.N[state,action] += 1
        delta = reward + self.gamma * np.max(self.Q[next_state,:]) - self.Q[state, action]
        l_rate = 1/self.N[state,action]
        self.Q[state,action] = self.Q[state,action] + l_rate* delta
    

In [0]:
import numpy as np
np.append(np.zeros(256),4)

In [0]:
# --------------
# Point 1
# --------------
env = CliffWalk(proba_succ=0.98)
# Number of Q learning steps
max_steps = int(1e5)  
# max_steps = 10

Q0 = np.zeros((env.Ns, env.Na))
# Use the previous code to verify the correctness of q learning
Q_opt, pi_opt, _ = value_iteration(Q0, env, epsilon=1e-8)

epsilon = 0.01
ql = QLearning(env, epsilon)

# main algorithmic loop
norm_values = []
rewards = [0]
t = 0
while t < max_steps:
    state = env.state
    greedy_policy = np.argmax(ql.Q, axis=1)
    action = ql.sample_action(state, greedy_policy)
    next_step, reward, done, info = env.step(action)
    norm_values.append(np.abs(ql.Q - Q_opt).mean())
    rewards.append(rewards[-1] + reward)
    
    ql.update(state, action, next_state, reward)
    t = t + 1


    
print(env.render())
print("optimal policy: ", pi_opt)
greedy_policy = np.argmax(ql.Q, axis=1)
print("learnt policy:", greedy_policy)


plt.figure(1)
plt.plot(norm_values)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Q-learning: Convergence of Q")

plt.figure(2)
plt.plot(rewards)
plt.xlabel('Iteration')
plt.ylabel('Cumulative Reward')
plt.title("Sum of Rewards")



# how confident are you in the performance of the algorithm? maybe a single run is not enough


Not very confident because the error is still high after 'max_step' iterations.
This is due to the fact that there are some states that were not explored that 
much and then were not updated to know what will be the optimal policy. 
The Robbins-Monro conditions impose that all states have to be visited
 infinitely often in order to converge towards the optimal Q and therefore
  the optimal value function.
If the MDP doesn't explore well all the states, the algorithm will converge
 to the optimal policy only if the initial state distribution is large 
 in order to start from all the states.

 In addition, the optimal policy is to go along the cliff. But the 
 epsilon-greedy algorithm will allow us to explore differents states and then 
 to fall from the cliff. So the best policy could not be optimal and we will 
 prefer to visit farthest states from the cliff than taking risks and fall. 

